In [1]:
import os
import json
import re
import shutil
input_folder = "encoded"
output_folder = "done"
import torch
import torch.nn.functional as F


In [13]:
# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


164
<class 'str'>
encoded\adder11.txt


In [14]:
# ground_truth_labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
ground_truth_labels = [0,1,2,3,4,5,6,7,8]
labels_torch = torch.tensor(ground_truth_labels)

# One-hot encode the labels
one_hot_labels = F.one_hot(labels_torch, num_classes=9)

print(one_hot_labels[0])

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0])


In [15]:
labels_dict = dict()
def label_verilog_file(file_name):
    label_mapping = {
        'adder': 0, 'ALU': 1, 'comparator': 2, 'decoder': 3,
        'encoder': 4, 'mult': 5, 'mux': 6, 'pe': 7, 'sub': 8
    }
    
    pattern = r"([a-zA-Z]+)(\d+)?"
    match = re.match(pattern, file_name)
    if match:
        base_name = match.group(1)
        if base_name in label_mapping:
            labels_dict[(base_name, label_mapping[base_name])] = labels_dict.get((base_name, label_mapping[base_name]), 0) + 1
            return one_hot_labels[label_mapping[base_name]].tolist()
        
    return None

def add_label_to_verilog_file(input_file_path, output_folder):
    if input_file_path.endswith('.txt'):
        with open(input_file_path, "r") as file:
            loaded_data = json.load(file)
            # Convert string elements to integers
            temp = []
            for sublist in loaded_data[0]:
                # int_list = [int(char) for char in sublist[1]]
                # temp.append(int_list)
                temp.append([sublist[0], sublist[2]])
                # sublist.pop(2)
                # sublist.pop(1)
            loaded_data[0] = temp
            label = label_verilog_file(os.path.basename(input_file_path))
            if label is not None and [label] not in loaded_data:  # Check if label already exists
                loaded_data.append([label])  # Add label as the third list
                output_file_path = os.path.join(output_folder, os.path.basename(input_file_path))
                with open(output_file_path, "w") as output_file:
                    json.dump(loaded_data, output_file)
                return True
    return False



# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process files in the input folder
for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)
    if os.path.isfile(file_path):
        if not add_label_to_verilog_file(file_path, output_folder):
            print(f"Failed to label: {file_path}")



In [17]:
failed_files = []

def get_files_in_folder(input_folder):
    file_list = []
    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        if os.path.isfile(file_path):
            file_list.append(file_path)
    return file_list

# Replace 'input_folder' with the path to the folder you want to read files from
verilog_files = get_files_in_folder(output_folder)
print(len(verilog_files))


# for v in verilog_files:
#     with open(v, "r") as file:
#         loaded_data = json.load(file)
#         # print(loaded_data[0])
        
#         for node in loaded_data[0]:
#             if len(node) != 4:
#                 failed_files.append(v)
#                 break
            
#how to save list in a txt file


164


# reduce classes: 
    -Decoder
    -Encoder
    -Mux

In [59]:
print(failed_files)
if failed_files:
    with open("failed_files.txt", "w") as output:
        output.write(str(failed_files))

[]


removed seg and bcd classes

In [18]:
labels_dict

{('adder', 0): 14,
 ('ALU', 1): 9,
 ('comparator', 2): 17,
 ('decoder', 3): 26,
 ('encoder', 4): 23,
 ('mult', 5): 24,
 ('mux', 6): 26,
 ('pe', 7): 16,
 ('sub', 8): 9}